In [2]:
import pandas as pd
import lightgbm as lgb

## Fraud

In [8]:
df_fraud = pd.read_csv("data/fraud_detection_train.csv")
print(df_fraud.shape)
df_fraud.head()

(200217, 53)


,visit_id,kdkc,dati2,typeppk,jkpst,umur,jnspelsep,los,cmg,severitylevel,...,proc63_67,proc68_70,proc71_73,proc74_75,proc76_77,proc78_79,proc80_99,proce00_e99,procv00_v89,label
0,1,1107,150,SB,P,64,2,0,F,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1303,200,C,L,45,1,9,E,3,...,0,0,0,0,0,0,4,0,0,1
2,3,1114,172,B,P,34,2,0,Q,0,...,0,0,0,0,0,0,0,0,0,1
3,4,601,90,SC,L,34,2,0,Q,0,...,0,0,0,0,0,0,0,0,0,1
4,5,1006,130,B,L,27,2,0,F,0,...,0,0,0,0,0,0,0,0,0,1


## Cost

In [13]:
df_cost = pd.read_csv("data/case_cost_prediction_train.csv")
print(df_cost.shape)
df_cost.head()

(57971, 36)


,row_id,tglpelayanan,kddati2,tkp,peserta,a,b,c,cb,d,...,kp,kt,ku,s,sa,sb,sc,sd,case,unit_cost
0,1,2014-04-01 00:00:00,332,40,179530,0,0,1,0,1,...,0,0,0,0,0,0,0,0,266,3.597440e+06
1,2,2016-11-01 00:00:00,54,40,104782,0,1,0,0,0,...,0,0,1,0,0,0,1,1,2453,4.951008e+06
2,3,2016-05-01 00:00:00,323,30,280645,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1690,1.984208e+05
3,4,2018-11-01 00:00:00,318,40,178685,0,1,0,0,0,...,0,0,0,0,0,0,0,3,1321,4.008756e+06
4,5,2019-10-01 00:00:00,150,30,1199321,1,0,1,0,0,...,1,0,0,0,0,1,0,5,73056,3.072272e+05


In [14]:
df_cost.describe()

,row_id,kddati2,tkp,peserta,a,b,c,cb,d,ds,...,kp,kt,ku,s,sa,sb,sc,sd,case,unit_cost
count,57971.000000,57971.000000,57971.000000,5.797100e+04,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,57971.0,...,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,57971.000000,5.797100e+04
mean,28986.000000,246.423125,34.990081,3.562209e+05,0.041538,0.388574,0.788894,0.000380,0.281710,0.0,...,0.039899,0.004330,0.001829,0.004123,0.002018,0.228476,0.953028,0.644943,6539.418451,1.961092e+06
std,16734.930565,143.447935,5.000033,4.120323e+05,0.210390,0.660382,0.679786,0.019477,0.595284,0.0,...,0.210092,0.065659,0.042722,0.082443,0.044880,0.925357,2.225974,1.234201,17607.280021,1.889367e+06
min,1.000000,1.000000,30.000000,8.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000e+05
25%,14493.500000,125.000000,30.000000,1.127735e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,424.000000,2.336742e+05
50%,28986.000000,243.000000,30.000000,1.975800e+05,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1359.000000,6.547994e+05
75%,43478.500000,362.000000,40.000000,4.386935e+05,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,4583.000000,3.531702e+06
max,57971.000000,528.000000,40.000000,3.328509e+06,2.000000,8.000000,6.000000,1.000000,5.000000,0.0,...,2.000000,1.000000,1.000000,5.000000,1.000000,9.000000,25.000000,10.000000,333441.000000,2.690550e+07


In [15]:
df_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57971 entries, 0 to 57970
Data columns (total 36 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   row_id        57971 non-null  int64  
 1   tglpelayanan  57971 non-null  object 
 2   kddati2       57971 non-null  int64  
 3   tkp           57971 non-null  int64  
 4   peserta       57971 non-null  int64  
 5   a             57971 non-null  int64  
 6   b             57971 non-null  int64  
 7   c             57971 non-null  int64  
 8   cb            57971 non-null  int64  
 9   d             57971 non-null  int64  
 10  ds            57971 non-null  int64  
 11  gd            57971 non-null  int64  
 12  hd            57971 non-null  int64  
 13  i1            57971 non-null  int64  
 14  i2            57971 non-null  int64  
 15  i3            57971 non-null  int64  
 16  i4            57971 non-null  int64  
 17  kb            57971 non-null  int64  
 18  kc            57971 non-nu

In [16]:
df_cost.groupby("a").size()

a
0    55692
1     2150
2      129
dtype: int64

In [17]:
df_cost.groupby("b").size()

b
0    38787
1    17061
2     1499
3      393
4       26
5      113
6       33
7       51
8        8
dtype: int64

In [18]:
df_cost.groupby("tkp").size()

tkp
30    29043
40    28928
dtype: int64

In [22]:
df_cost.value_counts("kddati2")

kddati2
264    140
113    138
407    136
296    136
32     135
      ... 
517     23
518     10
459     10
527      7
479      2
Length: 490, dtype: int64